In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils.extmath import safe_sparse_dot

import numpy as np
import pandas as pd
import math

In [3]:
# Tokyo Race Data (Import data)
# race_data = pd.read_csv("./Downloads/Dataset 0-2.csv",  
#                   names=["Distance", "Altitude"])

# #Flanders Men's Data
# flanders_men_data = pd.read_csv("./Downloads/Flanders Men_s.csv",  
#                   names=["Distance", "Altitude"])

# #Flanders Women's Data
# flanders_women_data = pd.read_csv("./Downloads/Flanders_ Women.csv",  
#                   names=["Distance", "Altitude"])


In [105]:
# Define constants
A_m = 1.9 # male surface area
A_f = 1.6 # female surface area
M_m = 88.9 # male mass (kg)
M_f = 71.22 # female mass (kg)
K_a = 0.0108 # air resistance
K_r = 0.143 # rolling resistance
g = 0.00981 # gravitational constant
R = 0.0063 # recovery (s^-1)
MVC = 1000 # watt/newton
k = 0.0153 # fatigue (s^-1)
v_w = 1 # velocity wind
theta_w = 1 # wind direction
mu = 0.75 # coefficient of friction
P_eq = MVC * (R/(2*k))*(-1+(math.sqrt(1+4*(k/R))))

# To populate
theta_rider = [0,20,20]
gradient_rider = [-0.2,0.5,0.8]
force_arr = [10,20,30,40,50]
r_rider = [50,4000,200]
segment_len_arr = [34,53,12] 

In [177]:
# Male

stand_in_arr = []
# Quadratic solver to get new velocity given a particular applied force
def get_new_velocity(mass, prev_vel, force, i):
    
    prev_segment_len = segment_len_arr[i-2] if i > 1 else segment_len_arr[0]
    a = (1 / 2) * (mass * prev_vel / prev_segment_len)
    b = 0 - force
    c = (K_a * A_m * prev_vel * pow(prev_vel + (v_w * math.sin((math.pi / 2) - abs(theta_rider[i-1] - theta_w))), 2) 
    - (g * gradient_rider[i-1] * mass * prev_vel) + (K_r * mass * prev_vel) 
    - ((mass * pow(prev_vel,3)) / (2 * prev_segment_len)))
       
    d = (b**2 - 4*a*c)
    if d < 0: d = 0
    result = (-b + math.sqrt(d)) / (2 * a)
    return result

# Call first instance of recursion
def dfs():
    final_max = -10000000

    to_return = []
    for j in range(5):     
        gr = dfs_helper(force_arr[j], 1000, 10, 0, 0, 0, 1, [force_arr[j]]) 
        max_down_the_path = gr[0]
        op = gr[1]
        
        print("gr",gr)
        
        if max_down_the_path > final_max:
            final_max = max_down_the_path
            to_return = stand_in_arr
            
    return final_max, to_return
    
def dfs_helper(force, prev_f_max, prev_vel, prev_vel_avg, prev_force, r, i, arr):
    
    # Last force added
    if i == len(segment_len_arr):
        maxi = -10000000
        init_size = len(arr)
        # At last race segment, no more recursive calls. Just return the fastest possible average velocity
        for j in range(5):
            temp_arr = arr
            if (len(temp_arr) > init_size): temp_arr.pop(len(temp_arr)-1)
            new_f_max = prev_f_max - (k * prev_f_max * prev_force / MVC) + (R * (MVC - prev_f_max))
            new_vel = get_new_velocity(M_m, prev_vel, force_arr[j], i)
         
            if (check_constraints(force, new_f_max, prev_vel, new_vel, segment_len_arr[i-1], i)):
                if (((prev_vel_avg * (i-1)) + new_vel) / i) > maxi:
                    maxi = (((prev_vel_avg * (i-1)) + new_vel) / i) 
                    temp_arr.append(force_arr[j])
        stand_in_arr = temp_arr 
        print("Gdi",stand_in_arr)
        return (maxi, stand_in_arr)
    
    new_f_max = prev_f_max - (k * prev_f_max * prev_force / MVC) + (R * (MVC - prev_f_max))
    max_v_avg = -10000000
    to_return_arr = []
    init_size = len(arr)
    
    for j in range(5):
        temp_arr = arr
        if (len(temp_arr) > init_size): temp_arr.pop(len(temp_arr)-1)
       
        # Calculate new velocity given chosen force
        new_vel = get_new_velocity(M_m, prev_vel, force_arr[j], i)
        
        # If the new velocity and max force are within constraints, run DFS, if not, then go to next iteration.
        if (check_constraints(force_arr[j], new_f_max, prev_vel, new_vel, segment_len_arr[i-1], i)):
            new_vel_avg = new_vel if (i == 0) else ((prev_vel_avg * (i-1)) + new_vel) / i
            
            # Recursive call here
            temp_arr.append(force_arr[j])
            rec_res = dfs_helper(force_arr[j], new_f_max, new_vel, new_vel_avg, force, r_rider[i], i+1, temp_arr)
            yuh = rec_res[0]
          
            if yuh > max_v_avg: 
                max_v_avg = yuh
                to_return_arr = rec_res[1]
                
       # if not len(temp_arr) == 0: temp_arr.pop(len(temp_arr)-1)
    print(i,"quesch", to_return_arr)
    return (max_v_avg, to_return_arr)

# Check for our 4 constraints
def check_constraints(force, new_f_max, prev_vel, new_vel, prev_segment_len, t):
    sof_bool = (new_f_max >= P_eq) and (new_f_max <= MVC)
    r_prev = r_rider[0] if t < 2 else r_rider[t-2]
    prev_segment_len = segment_len_arr[t-2] if t > 2 else segment_len_arr[1]
    velocity_bool = ((force * math.sqrt(mu * g * r_rider[t-1])) - 
                (K_a * A_m * math.sqrt(mu * g * r_rider[t-1]) * pow((math.sqrt(mu * g * r_rider[t-1]) + v_w * 
                math.sin(abs(theta_rider[t-1] - theta_w))),2) - (g * gradient_rider[t-1] * M_m * 
                math.sqrt(mu * g * r_rider[t-1])) + (K_r *  math.sqrt(mu * g * r_rider[t-1]) * M_m) + 
                 (M_m * pow((math.sqrt(mu * g * r_rider[t-1]) - math.sqrt(mu * g * r_prev)),2) 
                  * math.sqrt(mu * g * r_rider[t-1]) / (2 * prev_segment_len)))) <= 0 # constrain r
    return (new_vel <= 45) and (new_vel >= 0) and sof_bool and velocity_bool

In [178]:
dfs()

Gdi [10, 10, 10, 50]
Gdi [10, 10, 10, 20, 50]
Gdi [10, 10, 10, 20, 30]
2 quesch [10, 10, 10]
1 quesch [10]
gr (10.790127406448244, [10])
Gdi [20, 10, 10, 50]
Gdi [20, 10, 10, 20, 50]
Gdi [20, 10, 10, 20, 30]
2 quesch [20, 10, 10]
1 quesch [20]
gr (10.790127406448244, [20])
Gdi [30, 10, 10, 50]
Gdi [30, 10, 10, 20, 50]
Gdi [30, 10, 10, 20, 30]
2 quesch [30, 10, 10]
1 quesch [30]
gr (10.790127406448244, [30])
Gdi [40, 10, 10, 50]
Gdi [40, 10, 10, 20, 50]
Gdi [40, 10, 10, 20, 30]
2 quesch [40, 10, 10]
1 quesch [40]
gr (10.790127406448244, [40])
Gdi [50, 10, 10, 50]
Gdi [50, 10, 10, 20, 50]
Gdi [50, 10, 10, 20, 30]
2 quesch [50, 10, 10]
1 quesch [50]
gr (10.790127406448244, [50])


(10.790127406448244, [])